In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b1_2/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 4000
start_epoch = 0
batch_ratio = 14
train_batch = 10
source_batch = train_batch * batch_ratio
test_batch = 200
lr = 0.04
schedule = [500, 2000, 3000]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style2/128/b1_2/to_pggan/self' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'
feature_list = []

# iterative training
feedback = 0
iter_time = []

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01

# self-training
logsoftmax = nn.LogSoftmax(dim=-1)
softmax = nn.Softmax(dim=-1)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot')
source_train_dir = os.path.join(source_dir, 'train')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

source_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
source_loader = DataLoader(datasets.ImageFolder(source_train_dir, transform=val_aug),
                                batch_size=source_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
director_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                                      override_params={'dropout_rate':0.3, 'drop_connect_rate':0.3})
learner_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                                      override_params={'dropout_rate':0.3, 'drop_connect_rate':0.3})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    director_model.load_state_dict(torch.load(pretrained)['state_dict'])
    learner_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b1_2/checkpoint.pth.tar'


In [9]:
director_model.to('cuda')
learner_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in learner_model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
for param in director_model.parameters():
    param.requires_grad = False
director_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [11]:
director_model_weights = {}
for name, param in director_model.named_parameters():
    director_model_weights[name] = param.detach()

In [12]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [13]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5*torch.norm(param - director_model_weights[name])**2
    return sp_loss

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(learner_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(learner_model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    learner_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, source_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    global feature_list
    student_model.train()
    teacher_model.eval()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)
        
        dataiter = iter(source_loader)
        source_inputs, source_targets = next(dataiter)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            source_inputs, source_targets = source_inputs.cuda(), source_targets.cuda()
            
        # cutmix
        prob_delta = cm_prob_init - cm_prob_low
        prob_step = epoch / (epochs+1) * prob_delta
        lam = cm_prob_init - prob_step

        rand_index = torch.randperm(inputs.size()[0]).cuda()
        st = source_targets[rand_index*batch_ratio]
        tt = targets[rand_index]
        rand_index = rand_index[st == tt]
        
        bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
        source_inputs[rand_index*batch_ratio, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
        
         # label-training
        outputs = student_model(inputs)
        
        # director
        director_outputs = teacher_model(source_inputs)
        loss_source = criterion(director_outputs, source_targets)
        
        adaptive_param = torch.tensor(0.).cuda()
        adaptive_param += loss_source.item()
        
        # learner learn
        learn_outputs = student_model(source_inputs)
        
        
        loss_main = criterion(outputs, targets)
        loss_sub = criterion(learn_outputs, source_targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
            
            
        # compute output
        loss = loss_main + 0.1*loss_sub + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
        print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{total:.4f} | Adaptive:{data:.4f} | SubLoss:{eta:.4f} | | SPLoss:{sp:.4f} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=adaptive_param, total=loss_main, eta=loss_sub, loss=losses.avg, sp=loss_sp, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
#         inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    # teacher feedback
    if epoch in iter_time:
        print("iterative training: feedback {}".format(epoch))
        director_model.load_state_dict(learner_model.state_dict())
        dicrector_model_weights = {}
        for name, param in director_model.named_parameters():
            director_model_weights[name] = param.detach()
    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, source_loader, director_model, learner_model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        with torch.no_grad():
            director_model.eval()
            test_loss, test_acc = test(val_target_loader, learner_model, criterion, epoch, use_cuda)
            source_loss, source_acc = test(val_source_loader, learner_model, criterion, epoch, use_cuda)

    
        logger.append([state['lr'], train_loss, test_loss, source_loss, train_acc, test_acc, source_acc])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : learner_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 4000] LR: 0.040000
1/20 | Loss:0.7531 | MainLoss:0.6569 | Adaptive:0.7025 | SubLoss:0.7049 | | SPLoss:0.0000 | top1:70.0
2/20 | Loss:0.8010 | MainLoss:0.7539 | Adaptive:0.6954 | SubLoss:0.6920 | | SPLoss:0.0000 | top1:45.0
3/20 | Loss:0.7970 | MainLoss:0.6951 | Adaptive:0.6907 | SubLoss:0.6892 | | SPLoss:0.0000 | top1:46.66666793823242
4/20 | Loss:0.7869 | MainLoss:0.6617 | Adaptive:0.7060 | SubLoss:0.7050 | | SPLoss:0.0001 | top1:52.5
5/20 | Loss:0.7895 | MainLoss:0.7053 | Adaptive:0.7061 | SubLoss:0.7052 | | SPLoss:0.0001 | top1:50.0
6/20 | Loss:0.7847 | MainLoss:0.6676 | Adaptive:0.7084 | SubLoss:0.7023 | | SPLoss:0.0002 | top1:53.333335876464844
7/20 | Loss:0.7860 | MainLoss:0.7016 | Adaptive:0.7037 | SubLoss:0.6973 | | SPLoss:0.0002 | top1:51.42857360839844
8/20 | Loss:0.7870 | MainLoss:0.7033 | Adaptive:0.7025 | SubLoss:0.6984 | | SPLoss:0.0003 | top1:51.25
9/20 | Loss:0.7881 | MainLoss:0.7070 | Adaptive:0.7096 | SubLoss:0.6965 | | SPLoss:0.0005 | top1:50.0
10/20 | L

13/20 | Loss:0.7668 | MainLoss:0.6943 | Adaptive:0.7072 | SubLoss:0.6905 | | SPLoss:0.0007 | top1:50.0
14/20 | Loss:0.7663 | MainLoss:0.6907 | Adaptive:0.7084 | SubLoss:0.6924 | | SPLoss:0.0007 | top1:52.14285659790039
15/20 | Loss:0.7663 | MainLoss:0.6966 | Adaptive:0.6990 | SubLoss:0.6934 | | SPLoss:0.0007 | top1:50.66666793823242
16/20 | Loss:0.7662 | MainLoss:0.6932 | Adaptive:0.7061 | SubLoss:0.6929 | | SPLoss:0.0006 | top1:50.625
17/20 | Loss:0.7660 | MainLoss:0.6936 | Adaptive:0.7084 | SubLoss:0.6929 | | SPLoss:0.0006 | top1:50.0
18/20 | Loss:0.7658 | MainLoss:0.6922 | Adaptive:0.7084 | SubLoss:0.6941 | | SPLoss:0.0006 | top1:51.11111068725586
19/20 | Loss:0.7666 | MainLoss:0.7106 | Adaptive:0.7013 | SubLoss:0.6947 | | SPLoss:0.0006 | top1:49.47368621826172
20/20 | Loss:0.7660 | MainLoss:0.6838 | Adaptive:0.7155 | SubLoss:0.6970 | | SPLoss:0.0006 | top1:50.5

Epoch: [5 | 4000] LR: 0.088000
1/20 | Loss:0.7755 | MainLoss:0.7048 | Adaptive:0.7156 | SubLoss:0.7002 | | SPLoss:0.0006 

7/20 | Loss:0.8461 | MainLoss:0.7053 | Adaptive:0.7108 | SubLoss:0.6866 | | SPLoss:0.0114 | top1:45.71428680419922
8/20 | Loss:0.8337 | MainLoss:0.6744 | Adaptive:0.7049 | SubLoss:0.6897 | | SPLoss:0.0134 | top1:47.5
9/20 | Loss:0.8246 | MainLoss:0.6745 | Adaptive:0.7013 | SubLoss:0.7048 | | SPLoss:0.0151 | top1:48.88888931274414
10/20 | Loss:0.8181 | MainLoss:0.6766 | Adaptive:0.6990 | SubLoss:0.7259 | | SPLoss:0.0164 | top1:50.0
11/20 | Loss:0.8262 | MainLoss:0.8204 | Adaptive:0.7014 | SubLoss:0.7321 | | SPLoss:0.0172 | top1:49.09090805053711
12/20 | Loss:0.8391 | MainLoss:0.8915 | Adaptive:0.6990 | SubLoss:0.7431 | | SPLoss:0.0175 | top1:47.500003814697266
13/20 | Loss:0.8380 | MainLoss:0.7395 | Adaptive:0.6942 | SubLoss:0.7461 | | SPLoss:0.0175 | top1:47.69230651855469
14/20 | Loss:0.8387 | MainLoss:0.7670 | Adaptive:0.6978 | SubLoss:0.7175 | | SPLoss:0.0177 | top1:47.14285659790039
15/20 | Loss:0.8327 | MainLoss:0.6756 | Adaptive:0.7025 | SubLoss:0.6930 | | SPLoss:0.0181 | top1:48


Epoch: [12 | 4000] LR: 0.160000
1/20 | Loss:0.7168 | MainLoss:0.6384 | Adaptive:0.6942 | SubLoss:0.7148 | | SPLoss:0.0265 | top1:70.0
2/20 | Loss:0.8103 | MainLoss:0.8155 | Adaptive:0.7025 | SubLoss:0.7258 | | SPLoss:0.0352 | top1:55.0
3/20 | Loss:0.8043 | MainLoss:0.6956 | Adaptive:0.7049 | SubLoss:0.7347 | | SPLoss:0.0429 | top1:56.66666793823242
4/20 | Loss:0.8255 | MainLoss:0.7950 | Adaptive:0.7227 | SubLoss:0.6482 | | SPLoss:0.0498 | top1:55.0
5/20 | Loss:0.8048 | MainLoss:0.6171 | Adaptive:0.7061 | SubLoss:0.7354 | | SPLoss:0.0575 | top1:58.0
6/20 | Loss:0.8042 | MainLoss:0.6967 | Adaptive:0.7061 | SubLoss:0.7246 | | SPLoss:0.0642 | top1:58.333335876464844
7/20 | Loss:0.8114 | MainLoss:0.7488 | Adaptive:0.7001 | SubLoss:0.7435 | | SPLoss:0.0724 | top1:57.14285659790039
8/20 | Loss:0.8004 | MainLoss:0.6231 | Adaptive:0.7002 | SubLoss:0.7078 | | SPLoss:0.0861 | top1:58.75
9/20 | Loss:0.8009 | MainLoss:0.7064 | Adaptive:0.6989 | SubLoss:0.6986 | | SPLoss:0.0940 | top1:57.7777786254

15/20 | Loss:6.5588 | MainLoss:0.7378 | Adaptive:0.7037 | SubLoss:0.7285 | | SPLoss:115.0167 | top1:42.66666793823242
16/20 | Loss:6.8781 | MainLoss:0.7095 | Adaptive:0.7072 | SubLoss:0.7067 | | SPLoss:108.7183 | top1:42.5
17/20 | Loss:7.1084 | MainLoss:0.7240 | Adaptive:0.7096 | SubLoss:0.6838 | | SPLoss:99.8676 | top1:42.352943420410156
18/20 | Loss:7.2488 | MainLoss:0.6323 | Adaptive:0.7108 | SubLoss:0.6811 | | SPLoss:89.2070 | top1:43.88888931274414
19/20 | Loss:7.3154 | MainLoss:0.6795 | Adaptive:0.6989 | SubLoss:0.7284 | | SPLoss:77.4729 | top1:44.73684310913086
20/20 | Loss:7.3229 | MainLoss:0.8431 | Adaptive:0.7061 | SubLoss:0.7178 | | SPLoss:65.3487 | top1:44.5

Epoch: [16 | 4000] LR: 0.160000
1/20 | Loss:6.1144 | MainLoss:0.6849 | Adaptive:0.7037 | SubLoss:0.7190 | | SPLoss:53.4444 | top1:60.0
2/20 | Loss:5.5203 | MainLoss:0.6204 | Adaptive:0.7108 | SubLoss:0.6811 | | SPLoss:42.2734 | top1:65.0
3/20 | Loss:5.0511 | MainLoss:0.8144 | Adaptive:0.7073 | SubLoss:0.6916 | | SPLoss

...

18/20 | Loss:0.8478 | MainLoss:0.8107 | Adaptive:0.7037 | SubLoss:0.7223 | | SPLoss:0.9069 | top1:51.11111068725586
19/20 | Loss:0.8478 | MainLoss:0.6806 | Adaptive:0.7014 | SubLoss:0.6999 | | SPLoss:0.9503 | top1:51.578948974609375
20/20 | Loss:0.8485 | MainLoss:0.6941 | Adaptive:0.7025 | SubLoss:0.6934 | | SPLoss:0.9597 | top1:52.0

Epoch: [86 | 4000] LR: 0.159865
1/20 | Loss:0.8802 | MainLoss:0.7156 | Adaptive:0.7097 | SubLoss:0.6833 | | SPLoss:0.9392 | top1:40.0
2/20 | Loss:0.8578 | MainLoss:0.6731 | Adaptive:0.6990 | SubLoss:0.7005 | | SPLoss:0.8908 | top1:50.0
3/20 | Loss:0.8733 | MainLoss:0.7504 | Adaptive:0.7143 | SubLoss:0.6704 | | SPLoss:0.8213 | top1:46.66666793823242
4/20 | Loss:0.8803 | MainLoss:0.7572 | Adaptive:0.7179 | SubLoss:0.6606 | | SPLoss:0.7351 | top1:45.0
5/20 | Loss:0.8729 | MainLoss:0.7056 | Adaptive:0.7002 | SubLoss:0.6988 | | SPLoss:0.6388 | top1:46.0
6/20 | Loss:0.8686 | MainLoss:0.7213 | Adaptive:0.7002 | SubLoss:0.6940 | | SPLoss:0.5390 | top1:45.00000381

12/20 | Loss:0.8301 | MainLoss:0.7853 | Adaptive:0.7001 | SubLoss:0.7426 | | SPLoss:0.2023 | top1:53.333335876464844
13/20 | Loss:0.8300 | MainLoss:0.7231 | Adaptive:0.7084 | SubLoss:0.7506 | | SPLoss:0.2224 | top1:53.07692337036133
14/20 | Loss:0.8323 | MainLoss:0.7598 | Adaptive:0.7061 | SubLoss:0.7214 | | SPLoss:0.2395 | top1:51.42857360839844
15/20 | Loss:0.8299 | MainLoss:0.6984 | Adaptive:0.7014 | SubLoss:0.6949 | | SPLoss:0.2528 | top1:50.0
16/20 | Loss:0.8294 | MainLoss:0.7221 | Adaptive:0.7060 | SubLoss:0.6880 | | SPLoss:0.2666 | top1:49.375
17/20 | Loss:0.8263 | MainLoss:0.6735 | Adaptive:0.7037 | SubLoss:0.6918 | | SPLoss:0.2794 | top1:50.0
18/20 | Loss:0.8331 | MainLoss:0.8441 | Adaptive:0.7097 | SubLoss:0.6819 | | SPLoss:0.2890 | top1:48.333335876464844
19/20 | Loss:0.8379 | MainLoss:0.8156 | Adaptive:0.6989 | SubLoss:0.7027 | | SPLoss:0.2941 | top1:46.842105865478516
20/20 | Loss:0.8360 | MainLoss:0.6943 | Adaptive:0.7073 | SubLoss:0.6908 | | SPLoss:0.2992 | top1:46.5

Ep

6/20 | Loss:0.7832 | MainLoss:0.6160 | Adaptive:0.6930 | SubLoss:0.7102 | | SPLoss:0.0466 | top1:48.333335876464844
7/20 | Loss:0.7944 | MainLoss:0.7799 | Adaptive:0.6918 | SubLoss:0.7192 | | SPLoss:0.0474 | top1:45.71428680419922
8/20 | Loss:0.7928 | MainLoss:0.7031 | Adaptive:0.7061 | SubLoss:0.6872 | | SPLoss:0.0488 | top1:46.25
9/20 | Loss:0.7929 | MainLoss:0.7170 | Adaptive:0.7143 | SubLoss:0.6777 | | SPLoss:0.0505 | top1:45.55555725097656
10/20 | Loss:0.7901 | MainLoss:0.6872 | Adaptive:0.7002 | SubLoss:0.6931 | | SPLoss:0.0527 | top1:47.0
11/20 | Loss:0.7875 | MainLoss:0.6841 | Adaptive:0.6942 | SubLoss:0.6928 | | SPLoss:0.0536 | top1:49.09090805053711
12/20 | Loss:0.7869 | MainLoss:0.6991 | Adaptive:0.7096 | SubLoss:0.7164 | | SPLoss:0.0558 | top1:49.16666793823242
13/20 | Loss:0.7849 | MainLoss:0.6730 | Adaptive:0.7262 | SubLoss:0.7663 | | SPLoss:0.0562 | top1:50.0
14/20 | Loss:0.7891 | MainLoss:0.7579 | Adaptive:0.7084 | SubLoss:0.7463 | | SPLoss:0.0557 | top1:49.285713195800

20/20 | Loss:0.8074 | MainLoss:0.6752 | Adaptive:0.6918 | SubLoss:0.6922 | | SPLoss:0.2793 | top1:50.5

Epoch: [97 | 4000] LR: 0.159822
1/20 | Loss:0.7755 | MainLoss:0.6784 | Adaptive:0.6966 | SubLoss:0.7030 | | SPLoss:0.2329 | top1:60.0
2/20 | Loss:0.7557 | MainLoss:0.6404 | Adaptive:0.7060 | SubLoss:0.7231 | | SPLoss:0.1887 | top1:65.0
3/20 | Loss:0.7988 | MainLoss:0.7930 | Adaptive:0.6978 | SubLoss:0.7156 | | SPLoss:0.1482 | top1:53.333335876464844
4/20 | Loss:0.7890 | MainLoss:0.6680 | Adaptive:0.7191 | SubLoss:0.7574 | | SPLoss:0.1118 | top1:55.0
5/20 | Loss:0.7828 | MainLoss:0.6721 | Adaptive:0.7108 | SubLoss:0.7325 | | SPLoss:0.0809 | top1:56.0
6/20 | Loss:0.7831 | MainLoss:0.7032 | Adaptive:0.7084 | SubLoss:0.7206 | | SPLoss:0.0562 | top1:55.000003814697266
7/20 | Loss:0.7837 | MainLoss:0.7115 | Adaptive:0.6918 | SubLoss:0.6919 | | SPLoss:0.0373 | top1:52.85714340209961
8/20 | Loss:0.7801 | MainLoss:0.6807 | Adaptive:0.7025 | SubLoss:0.6959 | | SPLoss:0.0249 | top1:52.5
9/20 | 

14/20 | Loss:0.7859 | MainLoss:0.6772 | Adaptive:0.7025 | SubLoss:0.7045 | | SPLoss:0.0105 | top1:50.71428680419922
15/20 | Loss:0.7842 | MainLoss:0.6871 | Adaptive:0.6989 | SubLoss:0.6954 | | SPLoss:0.0112 | top1:51.333335876464844
16/20 | Loss:0.7824 | MainLoss:0.6826 | Adaptive:0.7084 | SubLoss:0.6939 | | SPLoss:0.0118 | top1:53.125
17/20 | Loss:0.7826 | MainLoss:0.7118 | Adaptive:0.7049 | SubLoss:0.6910 | | SPLoss:0.0125 | top1:51.764705657958984
18/20 | Loss:0.7814 | MainLoss:0.6874 | Adaptive:0.7049 | SubLoss:0.6898 | | SPLoss:0.0126 | top1:51.66666793823242
19/20 | Loss:0.7803 | MainLoss:0.6864 | Adaptive:0.6966 | SubLoss:0.6952 | | SPLoss:0.0125 | top1:52.10526657104492
20/20 | Loss:0.7778 | MainLoss:0.6559 | Adaptive:0.6930 | SubLoss:0.6976 | | SPLoss:0.0123 | top1:53.0

Epoch: [101 | 4000] LR: 0.159805
1/20 | Loss:0.8568 | MainLoss:0.7827 | Adaptive:0.7096 | SubLoss:0.6832 | | SPLoss:0.0119 | top1:20.0
2/20 | Loss:0.8316 | MainLoss:0.7313 | Adaptive:0.6942 | SubLoss:0.6987 | 